In [2]:
#@title ENV Preparation/Hazırlıklar

!apt install pciutils neofetch nano
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /content/ComfyUI/
!pip install -r requirements.txt
%cd ./cusom_nodes/
!git clone https://github.com/kijai/ComfyUI-HunyuanVideoWrapper.git
!pip install -r ComfyUI-HunyuanVideoWrapper/requirements.txt
%cd /content/ComfyUI/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors -P ./models/text_encoders/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors -P ./models/text_encoders/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/clip_vision/llava_llama3_vision.safetensors -P ./models/clip_vision/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/vae/hunyuan_video_vae_bf16.safetensors -P ./models/vae/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_vae_fp32.safetensors -P ./models/vae/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_I2V_720_fixed_bf16.safetensors -P ./models/diffusion_models/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_I2V_720_fixed_fp8_e4m3fn.safetensors -P ./models/diffusion_models/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_I2V-Q3_K_S.gguf -P ./models/diffusion_models/
!git clone https://github.com/ltdrdata/ComfyUI-Manager ./custom_nodes/comfyui-manager
!pip install xformers


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  caca-utils chafa fonts-droid-fallback fonts-noto-mono fonts-urw-base35
  ghostscript gsfonts imagemagick imagemagick-6-common imagemagick-6.q16 jp2a
  libchafa0 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9
  libgs9-common libid3tag0 libidn12 libijs-0.35 libimlib2 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libpci3
  libsixel-bin libsixel1 libwmflite-0.2-7 netpbm pci.ids poppler-data toilet
  toilet-fonts w3m w3m-img
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  hunspell

In [3]:
#@title Cloudflared ile tunneling yaparak açma
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server


--2025-09-04 10:01:38--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.8.1/cloudflared-linux-amd64.deb [following]
--2025-09-04 10:01:38--  https://github.com/cloudflare/cloudflared/releases/download/2025.8.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/92cb7978-e346-4125-ab19-54eebccb08b0?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-04T10%3A42%3A57Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title iframe ile açma

import threading
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-09-04 11:17:37.096
** Platform: Linux
** Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/default/ComfyUI-Manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   5.2 seconds: /content/ComfyUI/custom_nodes/comfyui-manager

Checkpoint files will always be loaded safely.
Total VRAM 40507 MB, total RAM 85479 MB
pytorch version: 2.8.0+cu126
xformers version: 0.0.32.post2
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA A100-SXM4-40GB : cudaMallocAsync
Using xformers attention
Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
ComfyUI version: 0.3.57
ComfyUI frontend version:

<IPython.core.display.Javascript object>

to open it in a window you can open this link here:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

FETCH ComfyRegistry Data: 15/96
FETCH ComfyRegistry Data: 20/96
FETCH ComfyRegistry Data: 25/96
FETCH ComfyRegistry Data: 30/96
FETCH ComfyRegistry Data: 35/96
FETCH ComfyRegistry Data: 40/96
FETCH ComfyRegistry Data: 45/96
FETCH ComfyRegistry Data: 50/96
FETCH ComfyRegistry Data: 55/96
FETCH ComfyRegistry Data: 60/96
FETCH ComfyRegistry Data: 65/96
FETCH ComfyRegistry Data: 70/96
FETCH ComfyRegistry Data: 75/96
FETCH ComfyRegistry Data: 80/96
FETCH ComfyRegistry Data: 85/96
FETCH ComfyRegistry Data: 90/96
FETCH ComfyRegistry Data: 95/96
FETCH ComfyRegistry Data [DONE]
[ComfyUI-Manager] default cache updated: https://api.comfy.org/nodes
FETCH DATA from: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json [DONE]
[ComfyUI-Manager] All startup tasks have been completed.
got prompt
ImageResizeToBucket: Closest size = (800, 1152), closest ratio = 0.69444
got prompt
got prompt
got prompt
got prompt
got prompt
got prompt
got prompt
encoded latents shape torch